# Problem 2:
We have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [17]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests

from bs4 import BeautifulSoup
import xml


Scrape the List of postal codes of Canada

In [18]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [19]:
soup = BeautifulSoup(source, 'xml')

In [20]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [21]:
# Search all the postcode, borough, neighborhood 
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [22]:
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [24]:
df.shape

(103, 3)

Adding Longitude and Latitude

In [25]:
#Getting the files from csv file in my github since geocoder was unreliable
url2 = "https://raw.githubusercontent.com/aymiee/Toronto/master/Geospatial_Coordinates.csv"
#s=requests.get(url2).content
dfCoordinates = pd.read_csv(url2)
#print(dfCoordinates.loc[dfCoordinates['Postal Code'] == 'M3A'].Latitude)

df['Latitude'] = None
df['Longitude'] = None

for index, row in df.iterrows():  
    Latitude = dfCoordinates.loc[dfCoordinates['Postal Code'] == row['PostalCode']].Latitude
    Longitude = dfCoordinates.loc[dfCoordinates['Postal Code'] == row['PostalCode']].Latitude
    row['Latitude'] = float(Latitude)
    row['Longitude'] = float(Longitude)
    
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,43.7533
1,M4A,North York,Victoria Village,43.7259,43.7259
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,43.6543
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,43.7185
4,M7A,Queen's Park,Ontario Provincial Government,43.6623,43.6623
5,M9A,Etobicoke,Islington Avenue,43.6679,43.6679
6,M1B,Scarborough,"Malvern, Rouge",43.8067,43.8067
7,M3B,North York,Don Mills North,43.7459,43.7459
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,43.7064
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,43.6572


In [26]:
df.shape

(103, 5)